In [1]:
import numpy
import pandas
import json

from ipywidgets import widgets, interact
from matplotlib import pyplot as plt

from obi_auth import get_token
from entitysdk.client import Client
from entitysdk.models import SingleNeuronSimulation
from obi_notebook import get_projects
from obi_notebook import get_entities

## Reading in data

You will first have to authenticate to the platform. To do that, just click on the link after running the python cell below and follow instructions. Then, you will have to select in the drop-dow menu the project that was used to produce the single cell simulations you want to analyse.

In [2]:
token = get_token(environment="production", auth_mode="daf")
project_context = get_projects.get_projects(token)

Please open url in a different tab:  https://www.openbraininstitute.org/auth/realms/SBO/device?user_code=CURI-UUOA


Dropdown(description='Select:', options=(('Single Cell Project', {'id': '50280a36-44b9-45e7-8b7d-db220a203ce5'…

Then you can add the IDs of the single neuron simulations you want to analyse. Alternatively, you can select them from the drop-down menu after running the python cell below.

Once this is done, we'll be able to fetch the data from the database.

In [15]:

client = Client(
    project_context=project_context,
    environment="production",
    token_manager=token,
)

simulation_ids = [
    # put here the id you want to use as a string
    # you can add as many simulation IDs as you want
]
# Alternative: Select from a table of entities
if not simulation_ids:
    simulation_ids = get_entities.get_entities("single-neuron-simulation", token, simulation_ids,
                                            project_context=project_context,
                                            multi_select=True,
                                            page_size=100)

Output()

This will download the simulations:

In [16]:
simulation_paths = []
for sim_id in simulation_ids:
    simulation = client.get_entity(
        entity_type=SingleNeuronSimulation,
        entity_id=sim_id,
    )
    asset = client.download_assets(
        simulation,
        output_path="./",  # here you can put a repo for downloading the simulation
    ).one()
    simulation_paths.append(asset.path)

ANd this will read the data:

In [17]:
data_keys = ["x", "y"]

def entries_to_df(entries):
    data = []    
    for entry in entries:
        df = pandas.DataFrame(dict([(k, entry.pop(k)) for k in data_keys]))
        for k, v in entry.items():
            df[k] = v
        data.append(df)
    ret = pandas.concat(data, axis=0)
    cols = [_c for _c in ret.columns if _c not in data_keys]
    ret = ret.fillna("_NONE").set_index(cols)
    return ret

def read_list_of_entries(lst):
    data = pandas.concat([entries_to_df(_v) for _v in lst.values()], axis=0)
    return data

def read_sim_config_data(fid):
    cfg = json.load(fid)
    data_out = read_list_of_entries(cfg["simulation"])
    stim_out = entries_to_df(cfg["stimulus"])
    return data_out, stim_out

all_data = []; all_stim = []
for sim_path in simulation_paths:
    with open(sim_path) as fid:
        data, stim = read_sim_config_data(fid)
        all_data.append(data); all_stim.append(stim)


## Plot traces

This plots all traces contained in the simulation output file.
A single property / value pair can be selected and only the traces matching the value of the property will be plotted.

In [18]:
sim_count = widgets.Dropdown(
    options=numpy.arange(len(all_data)),
    description='Sim. run #')

fltr_sel = widgets.Dropdown(
    options=all_data[sim_count.value].index.names,
    description="Property"
)

fltr_val = widgets.Dropdown(
    options=all_data[sim_count.value].index.to_frame()[fltr_sel.value].drop_duplicates().values,
    description="Value"
)

def update_func(sim_count_val, fltr_sel_val, fltr_val_val):
    data = all_data[sim_count_val]
    counts = data.index.to_frame().apply(lambda _x: len(_x.drop_duplicates()), axis=0)
    if (counts > 1).sum() >= 1:
        for lvl in counts.index[counts <= 1]:
            data = data.droplevel(lvl)
    fltr_sel.options = data.index.names
    if fltr_sel_val not in fltr_sel.options:
        return
    fltr_val.options = data.index.to_frame()[fltr_sel_val].drop_duplicates().values
    if fltr_val_val not in fltr_val.options:
        return

    def _plot_func(df):
        if len(df) > 0:
            lbl = ",".join(["=".join(map(str, _v)) for _v in zip(df.index.names, df.index[0])])
            plt.plot(df["x"], df["y"], lw=0.5, label=lbl)

    idx = data.index.names.index(fltr_sel_val)
    id_order = [idx] + list(numpy.setdiff1d(range(len(data.index.names)), idx))
    data = data.reorder_levels(id_order)
    data = data.loc[fltr_val_val]
    data.groupby(list(data.index.names)).apply(_plot_func)
    plt.legend()

    plt.gca().set_xlabel("time (ms)")
    plt.gca().set_ylabel("mV")
    

ii = interact(update_func, sim_count_val=sim_count, fltr_sel_val=fltr_sel, fltr_val_val=fltr_val)

interactive(children=(Dropdown(description='Sim. run #', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=0),…

## Plot spike counts

This applies a primitive spike detector and counts the numbers of spikes in each traces.
The spike count is then plotted. It is indicated by the size and color of a circular marker. The x- and y-coordinates of the marker are determined by the values of user-selectable properties.

In [19]:
_str_spk_count = "__spike_count"
def simple_spike_counter(df, thresh=0):
    v = df["y"].values > thresh
    return pandas.Series([numpy.sum(~v[:-1] & v[1:])], index=[_str_spk_count])

all_spks = [data.groupby(list(data.index.names)).apply(simple_spike_counter) for data in all_data]

sim_count = widgets.Dropdown(
    options=numpy.arange(len(all_spks)),
    description='Sim. run #')

prop_x = widgets.Dropdown(
    options=all_spks[sim_count.value].index.names,
    description="x"
)

prop_y = widgets.Dropdown(
    options=all_spks[sim_count.value].index.names,
    description="y"
)

def column_to_ordinates(s_in):
    if s_in.dtype == object:
        categorical = pandas.Categorical(s_in)
        ordinates = categorical.codes + 0.2 * (numpy.random.rand(len(categorical)) - 0.5)
        ticks = numpy.arange(len(categorical.categories))
        ticklabels = list(categorical.categories)
        return ordinates, ticks, ticklabels
    return s_in.values, numpy.unique(s_in.values), numpy.unique(s_in.values)

def update_func(sim_count_val, x_val, y_val):
    spks = all_spks[sim_count_val]
    counts = spks.index.to_frame().apply(lambda _x: len(_x.drop_duplicates()), axis=0)
    if (counts > 1).sum() >= 2:
        for lvl in counts.index[counts <= 1]:
            spks = spks.droplevel(lvl)
    prop_x.options = spks.index.names
    if x_val not in prop_x.options:
        return
    prop_y.options = spks.index.names
    if y_val not in prop_y.options:
        return
    
    spks = spks.reset_index()
    x, xticks, xticklabels = column_to_ordinates(spks[x_val])
    y, yticks, yticklabels = column_to_ordinates(spks[y_val])
    z = spks[_str_spk_count]

    plt.colorbar(plt.scatter(x, y, c=z, s=z+1), label="Spike count")
    plt.gca().set_xticks(xticks); plt.gca().set_xticklabels(xticklabels)
    plt.gca().set_yticks(yticks); plt.gca().set_yticklabels(yticklabels)
    plt.gca().set_xlabel(x_val)
    plt.gca().set_ylabel(y_val)
    


ii = interact(update_func, sim_count_val=sim_count, x_val=prop_x, y_val=prop_y)

interactive(children=(Dropdown(description='Sim. run #', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=0),…